# Self study 1

Self studies should be solved individually, or in small groups of 2-3 students. There is no hand-in of your solutins to the self studies. However, you can bring your solutions to the exam, and use them as the basis for your answers to the exam questions.

In this self-study we construct a simple crawler. Concretely, you should: 

* Select about 5 seed urls, e.g. homepages of universities, e-commerce sites, or similar

* Start crawling from these seeds. Define a strategy for selecting the next url to be crawled. What kind of prioritization (if any) is embodied in your strategy?

* Make sure you obey the robots.txt file, and make ensure that at least 2 seconds elapse between requests to the same host

* Stop when you have crawled approx. 1000 pages

* For each crawled page, save the url and the text string contained in the 'title' element of the document (we do not want to handle the full text of the pages at this point).

* You can repeat this several times, using different seed sets and/or prioritization strategies.

The following two self studies will extend the work that you do in this self study.

The following introduces a few helpful libraries and essential functions. You can use these methods, or use other tools that you are already familiar with and/or prefer to work with. 

A simple crawler implementation can be based on the 'requests' package [https://requests.readthedocs.io/en/master/](https://requests.readthedocs.io/en/master/) for retrieving html documents, and the BeautifulSoup parser https://www.crummy.com/software/BeautifulSoup/bs4/doc/ for parsing the html.

In [6]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from urllib.robotparser import RobotFileParser
from urllib.parse import urlparse
from datetime import datetime, timedelta
from itertools import count
import random


Let's start crawling at https://www.aau.dk/ . We first retrieve the robots.txt file and check whether we are allowed to crawl the top-level url:

In [7]:
rp=RobotFileParser()
rp.set_url("https://www.aau.dk")
rp.read()
print(rp.can_fetch("*","https://www.aau.dk"))

True


We can now get the html using the requests package, which returns a response object:

In [ ]:
r=requests.get('https://www.aau.dk/')
print(type(r))

A basic view of the contents is accessible via the content attribute:

In [ ]:
r.content

For serious parsing, we can use the BeautifulSoup html parser:

In [5]:
r_parse = BeautifulSoup(r.text, 'html.parser')
print(r_parse.prettify())

NameError: name 'r' is not defined

We can get the title:

In [4]:
print(r_parse.find('title'))
print(r_parse.find('title').string)

NameError: name 'r_parse' is not defined

Importantly, we can get all the links on the page. The following also illustrates the sleep() function to implement time delays (the following will take a while to complete; use the "interrupt kernel" button to terminate early):

In [ ]:
for a in r_parse.find_all('a'):
    sleep(1)
    print(a['href'])

In [16]:
url = 'https://www.aau.dk/uddannelser/optagelse/kandidat/ledige-studiepladser-2022'
parsed = urlparse(url)
print(parsed)
newurl = parsed.scheme + '://' + parsed.netloc
print(newurl)

ParseResult(scheme='https', netloc='www.aau.dk', path='/uddannelser/optagelse/kandidat/ledige-studiepladser-2022', params='', query='', fragment='')
https://www.aau.dk


In [42]:
seeds = ['https://www.aau.dk', 'https://www.dr.dk', 'https://www.tv2.dk', 'https://www.bt.dk', 'https://www.mit.edu']
index_arr = {}
crawled_links = []
frontier = []
frontqueue = {
    'one' : [],
    'two' : [],
    'three' : []
}

back_queue = {}
prio_heap = {}

for url in seeds :
    prio_heap[url] = datetime.now()
    back_queue[url] = []
sleep(2)

def get_base_url(url):
    parsed = urlparse(url)
    baseUrl = parsed.scheme + '://' + parsed.netloc
    return baseUrl
def fill_back_queue():
    arr = []
    if(len(frontqueue['one']) != 0):
        arr = frontqueue['one']
        frontqueue['one'] = []
    elif(len(frontqueue['two']) != 0):
        arr = frontqueue['two']
        frontqueue['two'] = []
    elif(len(frontqueue['three']) != 0):
        arr = frontqueue['three']
        frontqueue['three'] = []

    for url in arr:
        if (get_base_url(url) in back_queue.keys()):
            back_queue[get_base_url(url)].append(url)
            prio_heap[get_base_url(url)] = datetime.now()
        else:
            back_queue[get_base_url(url)] = [url]
            prio_heap[get_base_url(url)] = datetime.now()

def get_url():
    #This should be based on a heap but :shrugeg:
    viable_urls = [key for (key, value) in prio_heap.items() if value <= datetime.now() + timedelta(seconds=2)]

    randomUrl = random.choice(viable_urls)
    url = ""
    if(len(back_queue[randomUrl]) != 0):
        url = back_queue[randomUrl].pop()
    else:
        fill_back_queue()
        return get_url()
    crawled_links.append(randomUrl)
    return url

def fetch(url):
    rp.set_url(get_base_url(url))
    rp.read()
    if (True):#rp.can_fetch("*", url)):
        r=requests.get(url)
        r_parse = BeautifulSoup(r.text, 'html.parser')
        return r_parse
    else:
        return 0

def index(doc, url):
    title = doc.find('title')
    if(title):
        if url not in index_arr.keys():
            print(title.string)
            index_arr[url] = title.string
    else:
        print('no title')

def extract_urls(doc, url):
    href_arr = [] 
    for a in doc.find_all('a', href=True):
        link = a['href']
        if(link.startswith('https://www') and not link.startswith('https://www.google.com')):
            if (link not in frontier and link not in href_arr and link not in crawled_links):
                href_arr.append(link)
        #else:
        #    comb_url = get_base_url(url) + link
        #    if (comb_url not in frontier and comb_url not in href_arr and comb_url not in crawled_links):
        #        href_arr.append(comb_url)
    return href_arr

def add_to_frontier(url_list):
    #To make some checks easier this is added
    for url in url_list:
        frontier.append(url)
        slash_count = url.count('/')
        if (slash_count > 5):
            frontqueue['three'].append(url)
        elif(slash_count > 3):  
            frontqueue['two'].append(url)
        else:
            frontqueue['one'].append(url)

add_to_frontier(seeds)

i = 0
while (len(back_queue) != 0):
    i += 1
    url = get_url()
    print(url)
    doc = fetch(url)
    if (doc):
        index(doc, url)
        add_to_frontier(extract_urls(doc, url))
    if(i > 100):
        break

https://www.bt.dk
B.T. Nyheder | Læs nyhederne på bt.dk
https://www.tv2.dk
TV 2 - bedst pÃ¥ breaking og live
https://www.dr.dk
DR | Nyheder - Breaking - TV - Radio
https://www.aau.dk
AAU - Viden for verden - Aalborg Universitet
https://www.facebook.com/ditbt
B.T. - Startside
https://www.search.aau.dk?site=www.aau.dk&locale=da&mobile=false
AAU søg - Aalborg Universitet
https://www.en.aau.dk/
Aalborg University - Knowledge for the World
https://www.bt.dk/cookiedeklaration
Cookiedeklaration
https://www.ansatte.aau.dk
for ansatte
https://www.bt.dk/fodboldfm
Fodbold FM
https://www.update.aau.dk/
AAU UPDATE
https://www.kvalitetssikring.aau.dk
Uddannelseskvalitet på Aalborg Universitet
https://www.aub.aau.dk/
Aalborg Universitetsbibliotek
https://www.dr.dk/etik-og-rettelser
Etik og rettelser | DR
https://www.dr.dk/presse
Presse | DR
https://www.en.search.aau.dk/?locale=en
AAU Search - Aalborg University
https://www.instagram.com/aaustudieliv
Aalborg Universitet (@aaustudieliv) • Instagram pho

In [43]:
import nltk
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
import codecs

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [44]:
for key in index_arr.keys():
    print(f"{key}: {index_arr[key]}")

https://www.bt.dk: B.T. Nyheder | Læs nyhederne på bt.dk
https://www.tv2.dk: TV 2 - bedst pÃ¥ breaking og live
https://www.dr.dk: DR | Nyheder - Breaking - TV - Radio
https://www.aau.dk: AAU - Viden for verden - Aalborg Universitet
https://www.facebook.com/ditbt: B.T. - Startside
https://www.search.aau.dk?site=www.aau.dk&locale=da&mobile=false: AAU søg - Aalborg Universitet
https://www.en.aau.dk/: Aalborg University - Knowledge for the World
https://www.bt.dk/cookiedeklaration: Cookiedeklaration
https://www.ansatte.aau.dk: for ansatte
https://www.bt.dk/fodboldfm: Fodbold FM
https://www.update.aau.dk/: AAU UPDATE
https://www.kvalitetssikring.aau.dk: Uddannelseskvalitet på Aalborg Universitet
https://www.aub.aau.dk/: Aalborg Universitetsbibliotek
https://www.dr.dk/etik-og-rettelser: Etik og rettelser | DR
https://www.dr.dk/presse: Presse | DR
https://www.en.search.aau.dk/?locale=en: AAU Search - Aalborg University
https://www.instagram.com/aaustudieliv: Aalborg Universitet (@aaustudieliv

In [45]:
import nltk
nltk.download('punkt')
from nltk.stem.snowball import SnowballStemmer
import codecs

dstemmer=SnowballStemmer("danish")
a_file = codecs.open("stopord.txt", "r", "utf-8")
file_contents = a_file.read()
contents_split = file_contents.splitlines()
stop_words = []
for word in contents_split:
    stop_words.append(dstemmer.stem(word))
extra_words = ["|", ",", ".", "-", "!"]
for word in extra_words:
    stop_words.append(word)

for word in stop_words:
    print(word)
a_file.close()

ad
af
aldr
alen
all
all
alligevel
alt
altid
and
and
andr
at
bag
bar
beg
bl.a.
bland
blev
bliv
bliv
burd
bør
ca.
da
de
dem
den
den
den
der
dereft
der
derfor
derfra
deri
dermed
derpå
derved
det
det
dig
din
din
dis
dit
dog
du
eft
egen
ej
ell
ell
en
end
endnu
ene
enest
enhv
ens
ent
er
et
f.eks.
far
fem
fik
fir
fler
flest
flest
for
foran
fordi
for
fra
fx
få
får
før
først
gennem
gjord
gjort
god
godt
gør
gør
gør
ham
han
han
har
havd
hav
hej
hel
hel
helt
hen
hend
hend
henov
her
hereft
heri
hermed
herpå
hos
hun
hvad
hvem
hver
hvilk
hvilk
hvilk
hvis
hvor
hvordan
hvoreft
hvorfor
hvorfra
hvorh
hvori
hvorimod
hvornår
hvorved
i
igen
igennem
ikk
imellem
imen
imod
ind
indtil
ing
int
ja
jeg
jer
jer
jo
kan
kom
kom
kom
kun
kun
lad
lang
lav
lav
lav
lidt
lig
ligesom
lil
læng
man
mand
mang
med
meg
mellem
men
men
mer
mest
mig
min
mindr
mindst
min
mit
mod
må
måsk
ned
nej
nem
ni
nog
nogensind
nog
nogl
nok
nu
ny
nyt
når
nær
næst
næst
og
også
okay
om
omkring
op
os
ott
over
overalt
pga.
på
sam
sam
se
sek
selv
sel

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [63]:
stemmed_index = {}
for key in index_arr.keys():
    tokens = nltk.word_tokenize(index_arr[key])
    local_tokens = []
    for token in tokens:
        if token not in stop_words:
            local_tokens.append(token)
    stemmed_index[key] = local_tokens
            
for key in stemmed_index.keys():
    print(f"{key}: {stemmed_index[key]}")

https://www.bt.dk: ['B.T', 'Nyheder', 'Læs', 'nyhederne', 'bt.dk']
https://www.tv2.dk: ['TV', '2', 'bedst', 'pÃ¥', 'breaking', 'live']
https://www.dr.dk: ['DR', 'Nyheder', 'Breaking', 'TV', 'Radio']
https://www.aau.dk: ['AAU', 'Viden', 'verden', 'Aalborg', 'Universitet']
https://www.facebook.com/ditbt: ['B.T', 'Startside']
https://www.search.aau.dk?site=www.aau.dk&locale=da&mobile=false: ['AAU', 'søg', 'Aalborg', 'Universitet']
https://www.en.aau.dk/: ['Aalborg', 'University', 'Knowledge', 'the', 'World']
https://www.bt.dk/cookiedeklaration: ['Cookiedeklaration']
https://www.ansatte.aau.dk: ['ansatte']
https://www.bt.dk/fodboldfm: ['Fodbold', 'FM']
https://www.update.aau.dk/: ['AAU', 'UPDATE']
https://www.kvalitetssikring.aau.dk: ['Uddannelseskvalitet', 'Aalborg', 'Universitet']
https://www.aub.aau.dk/: ['Aalborg', 'Universitetsbibliotek']
https://www.dr.dk/etik-og-rettelser: ['Etik', 'rettelser', 'DR']
https://www.dr.dk/presse: ['Presse', 'DR']
https://www.en.search.aau.dk/?locale=en:

In [64]:
#used to give numbers
numbered_index = {}
i = 1
for url in stemmed_index.keys():
    stemmed_index[url] = {'tokens' : stemmed_index[url], 'id' : i}
    numbered_index[i] = url
    i += 1

In [65]:
for key in stemmed_index.keys():
    print(f"{key}: {stemmed_index[key]['tokens']} [{stemmed_index[key]['id']}]")

https://www.bt.dk: ['B.T', 'Nyheder', 'Læs', 'nyhederne', 'bt.dk'] [1]
https://www.tv2.dk: ['TV', '2', 'bedst', 'pÃ¥', 'breaking', 'live'] [2]
https://www.dr.dk: ['DR', 'Nyheder', 'Breaking', 'TV', 'Radio'] [3]
https://www.aau.dk: ['AAU', 'Viden', 'verden', 'Aalborg', 'Universitet'] [4]
https://www.facebook.com/ditbt: ['B.T', 'Startside'] [5]
https://www.search.aau.dk?site=www.aau.dk&locale=da&mobile=false: ['AAU', 'søg', 'Aalborg', 'Universitet'] [6]
https://www.en.aau.dk/: ['Aalborg', 'University', 'Knowledge', 'the', 'World'] [7]
https://www.bt.dk/cookiedeklaration: ['Cookiedeklaration'] [8]
https://www.ansatte.aau.dk: ['ansatte'] [9]
https://www.bt.dk/fodboldfm: ['Fodbold', 'FM'] [10]
https://www.update.aau.dk/: ['AAU', 'UPDATE'] [11]
https://www.kvalitetssikring.aau.dk: ['Uddannelseskvalitet', 'Aalborg', 'Universitet'] [12]
https://www.aub.aau.dk/: ['Aalborg', 'Universitetsbibliotek'] [13]
https://www.dr.dk/etik-og-rettelser: ['Etik', 'rettelser', 'DR'] [14]
https://www.dr.dk/pres

In [66]:
simple_inverted_index = {}

for url in stemmed_index.keys():
    for token in stemmed_index[url]['tokens']:
        if token in simple_inverted_index.keys():
            simple_inverted_index[token].append(url)
        else:
            simple_inverted_index[token] = [url]

for key in simple_inverted_index.keys():
    print(f"{key}: {simple_inverted_index[key]}")
    

B.T: ['https://www.bt.dk', 'https://www.facebook.com/ditbt']
Nyheder: ['https://www.bt.dk', 'https://www.dr.dk']
Læs: ['https://www.bt.dk']
nyhederne: ['https://www.bt.dk']
bt.dk: ['https://www.bt.dk']
TV: ['https://www.tv2.dk', 'https://www.dr.dk']
2: ['https://www.tv2.dk']
bedst: ['https://www.tv2.dk']
pÃ¥: ['https://www.tv2.dk']
breaking: ['https://www.tv2.dk']
live: ['https://www.tv2.dk']
DR: ['https://www.dr.dk', 'https://www.dr.dk/etik-og-rettelser', 'https://www.dr.dk/presse', 'https://www.dr.dk/om-dr', 'https://www.dr.dk/om-dr', 'https://www.facebook.com/DRDanmarksRadio']
Breaking: ['https://www.dr.dk']
Radio: ['https://www.dr.dk']
AAU: ['https://www.aau.dk', 'https://www.search.aau.dk?site=www.aau.dk&locale=da&mobile=false', 'https://www.update.aau.dk/', 'https://www.en.search.aau.dk/?locale=en', 'https://www.design.aau.dk/', 'https://www.en.search.aau.dk/?site=www.en.search.aau.dk&locale=en&mobile=false', 'https://www.nyheder.aau.dk/', 'https://www.its.aau.dk/', 'https://www.